In [1]:
!pip install numpy
!pip install pandas
!pip install os
!pip install nltk

ERROR: Could not find a version that satisfies the requirement os (from versions: none)
ERROR: No matching distribution found for os


In [2]:
import numpy as np
import pandas as pd
import os
import sys
from nltk.stem import PorterStemmer
import re

In [3]:
link_drive = "/Users/nguyennamhai/HUST/Training phase Machine Learning lab 2023/Phase 1/Session 1/dataset/20news-bydate"

In [4]:
print(os.listdir(link_drive))

['.DS_Store', 'stop_words.txt', '20news-bydate-test', '20news-bydate-train']


In [5]:
train_dir = [link_drive + "/" + path for path in os.listdir(link_drive) if "train" in path][0]
test_dir = [link_drive + "/" + path for path in os.listdir(link_drive) if "test" in path][0]
stop_words_dir = [link_drive + "/" + path for path in os.listdir(link_drive) if "stop_words" in path][0]
print(train_dir)
print(test_dir)
print(stop_words_dir)

/Users/nguyennamhai/HUST/Training phase Machine Learning lab 2023/Phase 1/Session 1/dataset/20news-bydate/20news-bydate-train
/Users/nguyennamhai/HUST/Training phase Machine Learning lab 2023/Phase 1/Session 1/dataset/20news-bydate/20news-bydate-test
/Users/nguyennamhai/HUST/Training phase Machine Learning lab 2023/Phase 1/Session 1/dataset/20news-bydate/stop_words.txt


In [6]:
# Remove duplicate words in stop_words.txt
def remove_duplicate_words_in_stop_words_files():
  with open(stop_words_dir, "r") as f:
    stop_words = f.read().splitlines()
  print(len(stop_words))
  stopwords = list()
  for word in stop_words:
    if word not in stopwords:
      stopwords.append(word)
  print(len(stopwords))
  with open(stop_words_dir, "w") as f:
    for word in stopwords:
      f.write(word)
      f.write('\n')

In [7]:
with open(stop_words_dir, "r") as f:
  stop_words = f.read().splitlines()
print(len(stop_words))

487


In [8]:
list_newsgroups = os.listdir(train_dir)
list_newsgroups.sort()
list_newsgroups = list_newsgroups[2:]
print(list_newsgroups)

['comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware', 'comp.windows.x', 'misc.forsale', 'rec.autos', 'rec.motorcycles', 'rec.sport.baseball', 'rec.sport.hockey', 'sci.crypt', 'sci.electronics', 'sci.med', 'sci.space', 'soc.religion.christian', 'talk.politics.guns', 'talk.politics.mideast', 'talk.politics.misc', 'talk.religion.misc']


In [ ]:
# Gather all data from all groups
stemmer = PorterStemmer()
def collect_data_from(parent_dir, newsgroup_list):
  data = []
  seen_filename = list()
  files = list()
  for group_id, newsgroup in enumerate(newsgroup_list):
    label = group_id
    dir_path = parent_dir + '/' + newsgroup + '/'
    for filename in os.listdir(dir_path):
      if os.path.isfile(dir_path + filename) and filename not in seen_filename:
        seen_filename.append(filename)
        files.append([filename, dir_path + filename])
    files.sort()

    for filename, filepath in files:
      with open(filepath, errors='ignore') as f:
        text = f.read().lower()
        # Remove stop words and stemming words
        words = [stemmer.stem(word) for word in re.split('\W+', text) if word not in stop_words]
        content = ' '.join(words)
        assert len(content.splitlines()) == 1
        data.append(str(label) + '<fff>' + filename + '<fff>' + content)
  return data  

data_train = collect_data_from(train_dir, list_newsgroups)
data_test = collect_data_from(test_dir, list_newsgroups)
data_full = data_train + data_test

# Write data to file with w+
with open(link_drive + "/20news_train_processed.txt", "w+") as f:
    f.write("\n".join(data_train))
with open(link_drive + "/20news_test_processed.txt", "w+") as f:
    f.write("\n".join(data_test))
with open(link_drive + "/20news_full_processed.txt", "w+") as f:
    f.write("\n".join(data_full))

In [ ]:
# Generate Dictionary(Bag of Words)
with open(link_drive + "/20news_full_processed.txt", errors='ignore') as f:
    lines = f.read().splitlines()
doc_count = dict()
corpus_size = len(lines)

for line in lines:
    features = line.split('<fff>')
    text = features[-1]
    words = text.split()
    for word in words:
        doc_count[word] = doc_count.get(word, 0) + 1
print(len(doc_count))

In [ ]:
# Compute IDF:
def compute_idf(doc_freq, corpus_size):
  assert doc_freq > 0
  return np.log10(corpus_size * 1 / doc_freq)

words_idfs = list()
for word, document_freq in doc_count.items():
  if document_freq > 10 and word.isdigit() == False:
    words_idfs.append([word, compute_idf(document_freq, corpus_size)])

words_idfs.sort(key=lambda word_idf: -word_idf[1])
# Write dictionary file 
with open(link_drive + "/words_idfs.txt", 'w+') as f:
    f.write('\n'.join([word + '<fff>' + str(idf) for word, idf in words_idfs]))

In [ ]:
# TF-IDF
print(0)
with open(link_drive + "/words_idfs.txt", errors='ignore') as f:
  lines = f.read().splitlines()

words_idfs = list()
words_IDs = dict()

for line in lines:
  l = line.split('<fff>')
  words_idfs.append([l[0], float(l[1])])
for index, (word, idf) in enumerate(words_idfs):
  words_IDs[word] = index
idfs = dict(words_idfs)
print(1)
with open(link_drive + "/20news_full_processed.txt", errors='ignore') as f:
  lies = f.read().splitlines()
print(2)
documents = list()
for lie in lies:
  l = lie.split('<fff>')
  documents.append([l[0], float(l[1]), l[2]])
  data_tf_idf = []
  for document in documents:
    label, doc_id, text = document
    words = [word for word in text.split() if word in idfs]
    word_set = list(set(words))
    max_term_freq = max([words.count(word) for word in word_set])
    words_tfidfs = []
    sum_squares = 0.0

    for word in word_set:
      term_freq = words.count(word)
      tf_idf_value = term_freq * (1. / max_term_freq) * idfs[word]
      words_tfidfs.append([words_IDs[word], tf_idf_value])
      sum_squares += tf_idf_value ** 2

  words_tfidfs_normalize = [str(index) + ':'+ str(tf_idf_value / np.sqrt(sum_squares)) for index, tf_idf_value in words_tfidfs]
  sparse_rep = ' '.join(words_tfidfs_normalize)
  data_tf_idf.append([label, doc_id, sparse_rep])
print(3)
# Write file data_tf_idf with abc
with open(link_drive + "/data_tf_idf.txt", 'w+') as f:
  result = []
  for num in range(len(data_tf_idf)):
    connect = "".join([str(data_tf_idf[num][0]), str(data_tf_idf[num][1]), data_tf_idf[num][2]])
    result.append(connect)
  f.write('\n'.join(result))